# Árboles de Decisión

Para esta práctica utilizaremos el dataset del Titanic (muy popular en Kaggle), que contiene los datos de los pasajeros del Titanic, famoso barco que se hundió en su primer viaje en 1912. Dicho dataset contiene los datos de los pasajeros así como información sobre si sobrevivieron o no. Generaremos algunos árboles de decisión y determinaremos cuál de ellos fue el modelo más preciso.

Descripción de los datos:

1. superviviente: El pasajero sobrevivió 0 = No, 1 = Si
2. clase_billete: Clase de camarote 1 = Primera clase, 2 = Segunda, 3 = Tercera clase (la más pobre)
3. genero: hombre/mujer
4. edad
5. n_hermanos_esposos: Número de hermanos o pareja a bordo del Titanic
6. n_hijos_padres: Número de hijos o padres a bordo del Titanic
7. precio_billete
8. puerto_salida: Puerto donde el pasajero tomó el barco (C=Cherbourg, Q=Queenstown, S=Southampton)

In [1]:
import pandas as pd

In [2]:
# Cargamos la tabla
datos = pd.read_csv("C:\\Users\\Luis Carlos\\Documents\\CSVs\\titanic.csv")

In [3]:
# Visualizamos la tabla
datos

,superviviente,clase_billete,genero,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,puerto_salida
0,0,3,hombre,22.0,1,0,7.2500,S
1,1,1,mujer,38.0,1,0,71.2833,C
2,1,3,mujer,26.0,0,0,7.9250,S
3,1,1,mujer,35.0,1,0,53.1000,S
4,0,3,hombre,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,hombre,27.0,0,0,13.0000,S
887,1,1,mujer,19.0,0,0,30.0000,S
888,0,3,mujer,NaN,1,2,23.4500,S
889,1,1,hombre,26.0,0,0,30.0000,C


Los algoritmos de creación de árboles están en el submódulo "sklearn.tree". Procedemos a cargar este módulo y también el de la validación cruzada.

In [4]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

Sci-Kit Learn nos proporciona modelos de árboles de decisión tanto para realizar regresiones lineales (utilizadas para predecir valores numéricos) como para regresiones logísticas o clasificaciones (utilizadas para predecir etiquetas).

Los modelos que contiene sk-learn son:

1. Para regresiones lineales: tree.DecisionTreeRegression()
2. Para regresiones logísticas o clasificaciones: tree.DecisionTreeClassifier()

En este caso utilizaremos el de clasificación para intentar predecir, en base a ciertos parámetros, si un pasajero sobrevivió o no.

In [5]:
# Creamos el modelo
arbol = tree.DecisionTreeClassifier()

Se debe realizar un preprocesamiento de los datos de la tabla en caso de que tengan valores nulos o faltantes. Para ello, consultamos esta información con la siguiente función:

In [6]:
datos.isnull().sum() 

superviviente           0
clase_billete           0
genero                  0
edad                  177
n_hermanos_esposos      0
n_hijos_padres          0
precio_billete          0
puerto_salida           2
dtype: int64

La columna "edad" tiene 177 valores nulos. Al ser una columna numérica, rellenaremos los datos faltantes con la media de los datos contenidos en esta columna. Por otro lado, la columna "puerto_salida" contiene 2 valores nulos, los cuales, al ser una columna categórica, se rellenará con la moda de los valores de ésta.

Comenzaremos por rellenar los valores faltantes de la columna categórica.

In [7]:
# Visualizamos los valores y la cantidad de ellos en esta columna
datos.puerto_salida.value_counts()

S    644
C    168
Q     77
Name: puerto_salida, dtype: int64

Rellenaremos con "S" los 2 valores faltantes, ya que es el valor que más se repite.

In [8]:
# Rellenamos los datos faltantes
datos.puerto_salida = datos.puerto_salida.fillna('S')

In [9]:
datos.puerto_salida.value_counts()

S    646
C    168
Q     77
Name: puerto_salida, dtype: int64

Podemos observar que efectivamente se han rellenado los valores faltantes de esta columna. Ahora procedemos a rellenar los nulos de la columna numérica.

In [10]:
# Rellenamos los datos faltantes
datos.edad = datos.edad.fillna(datos.edad.mean())

In [11]:
datos.isnull().sum() 

superviviente         0
clase_billete         0
genero                0
edad                  0
n_hermanos_esposos    0
n_hijos_padres        0
precio_billete        0
puerto_salida         0
dtype: int64

Vemos que hemos rellenado todos los valores nulos de nuestra tabla correctamente. Ahora convertiremos las variables categóricas a variables dummies y las agregaremos a nuestra tabla.

In [12]:
columnas_categoricas = ["genero", "puerto_salida"]

In [13]:
datos_categoricos = pd.get_dummies(datos[columnas_categoricas])

In [14]:
# Visualizamos las variables dummies
datos_categoricos.head()

,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,1,0,0,0,1
1,0,1,1,0,0
2,0,1,0,0,1
3,0,1,0,0,1
4,1,0,0,0,1


In [15]:
# Reemplazamos las variables dummies por las categóricas en nuestra tabla

pasajeros = (
    pd.concat([
        datos.drop(columnas_categoricas, axis=1),
        datos_categoricos
    ],axis=1
    )
)

In [16]:
# Visualizamos nuestra nueva tabla
pasajeros.head()

,superviviente,clase_billete,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,0,3,22.0,1,0,7.2500,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,1,0,0
2,1,3,26.0,0,0,7.9250,0,1,0,0,1
3,1,1,35.0,1,0,53.1000,0,1,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,0,1


## Primer modelo: Árbol normal

In [17]:
# Hacemos el ajuste del modelo, quitando la variable objetivo (superviviente)
arbol.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier()

In [18]:
# Determinamos mediante el criterio curva ROC qué tan efectivo fue el modelo
cross_val_score(arbol, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente, scoring="roc_auc", 
                cv=10).mean()

0.7620745550745551

Una funcionalidad interesante que tienen los arboles de decision en sklearn es que se pueden visualizar con Graphviz. Se puede exportar el árbol y abrirlo posteriormente con graphviz desde la terminal (o desde la página http://webgraphviz.com/ que renderiza archivos de graphviz), o también desde la página 
Graphviz Online - https://dreampuf.github.io.

In [19]:
# Generamos el archivo que ingresaremos en la página antes mencionada

import graphviz
tree.export_graphviz(arbol, out_file="arbol.dot")

<img src="arbol.jpg">

Después de ingresar en la página el código que se nos generó en la línea anterior, nos arrojó el siguiente árbol. Vemos que es un árbol extremadamente enorme, que ni siquiera cupo en el navegador. De cualquier forma, lo que nos interesa no es la visualización del árbol sino la efectividad del mismo, ver qué tan preciso fue.

Otra funcionalidad que tienen los árboles de decisión en sklearn es que nos dan una indicación de la importancia de cada variable en el modelo, almacenada en el atributo "feature_importances_". Calcula la importancia en función de la ganancia de información de cada variable, es decir, qué variables separan mejor las distintas clases.

In [20]:
arbol.feature_importances_

array([0.10817857, 0.26319392, 0.04733681, 0.01574422, 0.24264029,
       0.        , 0.30933519, 0.00337536, 0.        , 0.01019564])

In [21]:
dict(zip(
    pasajeros.drop("superviviente", axis=1),
    arbol.feature_importances_
))

{'clase_billete': 0.10817857458791365,
 'edad': 0.26319391768740075,
 'n_hermanos_esposos': 0.047336811047533935,
 'n_hijos_padres': 0.01574421531267729,
 'precio_billete': 0.24264029030071627,
 'genero_hombre': 0.0,
 'genero_mujer': 0.3093351886283387,
 'puerto_salida_C': 0.00337535806904541,
 'puerto_salida_Q': 0.0,
 'puerto_salida_S': 0.010195644366374062}

Se puede observar que el factor más determinante fue el de si el género de una persona era el de ser mujer. El segundo fue el de la edad y el tercero fue el de el precio del billete, por lo que se puede concluir que la frase "¡Mujeres y niños primero!" para salvar personas seguramente se aplicó en este caso.

## Segundo modelo: Árbol simple

Ahora, crearemos un árbol agregándole el parámetro "max_depth":La profundidad máxima del árbol. Definimos profundidad como el número de nodos que atraviesa una observación (cuantas "preguntas" se le hacen).

In [22]:
# Creamos el modelo
arbol_simple = tree.DecisionTreeClassifier(max_depth=3)

In [23]:
# Hacemos el ajuste
arbol_simple.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(max_depth=3)

In [24]:
# Exportamos el archivo
tree.export_graphviz(arbol_simple, out_file="arbol_simple.dot")

<img src="arbol_simple.jpg">

Vemos ahora que el árbol tuvo un diseño muchísimo más simplificado. Como lo comenté anteriormente, lo que nos interesa no es el diseño del árbol, sino lo preciso que es.

In [25]:
# Determinamos mediante el criterio curva ROC qué tan efectivo fue el modelo
cross_val_score(arbol_simple, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

0.8502134227428346

Recordemos que entre más próximo a 1 se encuentre el criterio de curva ROC, más preciso es el modelo, por lo que quiere decir que este modelo de árbol simple fue más preciso que el anterior.

## Tercer modelo: Árbol balanceado

También podemos crear un árbol agregándole el parámetro "class_weight": Para clases imbalanceadas, podemos pasar el string "balanced" para que sklearn genere pesos en función del número de muestras de cada clase.

In [26]:
# Creamos el modelo
arbol_balanceado = tree.DecisionTreeClassifier(max_depth=3, class_weight="balanced")

In [27]:
# Hacemos el ajuste
arbol_balanceado.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(class_weight='balanced', max_depth=3)

In [28]:
# Exportamos el archivo
tree.export_graphviz(arbol_balanceado, out_file="arbol_balanceado.dot")

<img src="arbol_balanceado.jpg">

In [29]:
# Determinamos mediante el criterio curva ROC qué tan efectivo fue el modelo
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

0.8615064652123475

Observamos que a comparación con el segundo modelo, éste mejoró un poquito más.

Así como tomamos el criterio de "curva ROC" para determinar qué tan preciso fue un modelo, también pudimos haber tomado el criterio de "precisión".

In [30]:
# Determinamos ahora con el modelo de precisión qué tan efectivo fue el modelo
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente,
                scoring="precision", cv=10).mean()

0.7411115910638909

Cabe recalcar que no puedes comparar un resultado obtenido con el criterio de precisión con uno obtenido con el criterio de curva ROC. Es decir, solamente podrás comparar los criterios de los modelos, curvas ROC con curvas ROC, o precisión con precisión.

## Cuarto modelo: Árbol aleatorio

Además del algoritmo CART para generar árboles, scikit-learn también proporciona una clase de arboles llamada ExtraTreeClassifier, o Extremely Random Trees (Árboles Extremadamente Aleatorios). En estos árboles, en lugar de seleccionar en cada nodo la párticion que proporciona la mayor ganancia de información, se decide una partición al azar.

In [31]:
# Creamos el modelo
arbol_aleatorio = tree.ExtraTreeClassifier(max_features=1)

In [32]:
# Hacemos el ajuste
arbol_aleatorio.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

ExtraTreeClassifier(max_features=1)

In [33]:
# Exportamos el archivo
tree.export_graphviz(arbol_aleatorio, out_file="arbol_aleatorio.dot")

<img src="arbol_aleatorio.jpg">

¡Podemos observar que este árbol fue incluso muchísimo más grande que el primero!

In [34]:
# Determinamos mediante el criterio curva ROC qué tan efectivo fue el modelo
cross_val_score(arbol_aleatorio, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente, scoring="roc_auc",
                cv=10).mean()

0.7592982202982202

Para este caso, los árboles aleatorios funcionan peor que los árboles de decisión por sí mismos.

En base a estos modelos, podemos concluir que el mejor de los 4 fue el tercero, es decir, el modelo de árbol balanceado.